In [9]:
#!pip install pathlib2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Tuple, Any
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score as accuracy, f1_score
from sklearn.preprocessing import scale
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib2 import Path
from os.path import join
import os
import re
import pickle
import yaml

**Yahoo Finance Setup**



Library Import

In [3]:
import yfinance as yf

**TA LIB setup**


In [4]:
#url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
#!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
#url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
#!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

**Fred API**

In [5]:
#!pip install fredapi
from fredapi import Fred
fred = Fred(api_key='X')

**Investing.com API**

In [ ]:
#!pip install investiny
#from investiny import historical_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.0
    Uninstalling pydantic-2.7.0:
      Successfully uninstalled pydantic-2.7.0


Helper **Functions**

In [6]:
def go_back_n_days(date_str, n):
    """
    This function takes a date string in the format "YYYY-MM-DD" and an integer n,
    and returns a new date string representing the date n days before the given date.

    Args:
    date_str (str): The input date string in the format "YYYY-MM-DD".
    n (int): The number of days to go back from the given date.

    Returns:
    str: A date string in the format "YYYY-MM-DD" of the new date.
    """
    # Parse the input date string
    date_format = "%Y-%m-%d"
    current_date = datetime.strptime(date_str, date_format).date()

    # Compute the date n days before
    new_date = current_date - timedelta(days=n)

    # Return the new date in the same string format
    return new_date.strftime(date_format)

**Primative Data Download**

In [6]:
INDEX = ["^DJI", "^IXIC", "^NYA", "^RUT", "^GSPC"]
start_date = "2009-12-31"
end_date = "2019-12-01"

data = {}

for index in INDEX:
    data[index] = yf.download(index, start=start_date, end=end_date, progress=False)

*V2 - Closing price* - Included

*V3 - Relative change of volume*

In [7]:

vol_relative_change = {}
for index, df in data.items():
    # Ensure volume is float for calculations
    df['Volume'] = df['Volume'].astype(float)
    # Calculate the relative change of volume (current volume divided by previous day's volume)
    df["Vol"] = (df['Volume'] / df['Volume'].shift(1)) - 1


*V4 to V14*

In [9]:
for index, df in data.items():
    # Calculate Momentum (MOM) for 1, 2, and 3 days before
    df['mom'] = df['Close'].pct_change(periods=1)  #NOT INCLUDED IN TABLE!!!**********
    df['mom1'] = df['mom'].shift(1)  # Shifted 1 day to get the return of 2 days ago
    df['mom2'] = df['mom'].shift(2)  # Shifted 2 days to get the return of 3 days ago
    df['mom3'] = df['mom'].shift(3)  # Shifted 3 days to get the return of 4 days ago


    # Calculate Rate of Change (ROC) for 5, 10, 15, and 20 days
    df['ROC_5'] = talib.ROC(df['Close'], timeperiod=5)
    df['ROC_10'] = talib.ROC(df['Close'], timeperiod=10)
    df['ROC_15'] = talib.ROC(df['Close'], timeperiod=15)
    df['ROC_20'] = talib.ROC(df['Close'], timeperiod=20)

    # Calculate Exponential Moving Average (EMA) for 10, 20, 50, and 200 days
    df['EMA_10'] = talib.EMA(df['Close'], timeperiod=10)
    df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
    df['EMA_50'] = talib.EMA(df['Close'], timeperiod=50)
    df['EMA_200'] = talib.EMA(df['Close'], timeperiod=200)

*V15 to V31*

In [10]:
for index, df in data.items():
  #V15
  df["DTB4WK"] = fred.get_series("DTB4WK", observation_start=start_date, observation_end=end_date)
  #V16
  df["DTB3"] = fred.get_series("DTB3", observation_start=start_date, observation_end=end_date)
  #V17
  df["DTB6"] = fred.get_series("DTB6", observation_start=start_date, observation_end=end_date)
  #V18
  df["DGS5"] = fred.get_series("DGS5", observation_start=start_date, observation_end=end_date)
  #V19
  df["DGS10"] = fred.get_series("DGS10", observation_start=start_date, observation_end=end_date)
  #V20
  df["DAAA"] = fred.get_series("DAAA", observation_start=start_date, observation_end=end_date)
  #V21
  df["DBAA"] = fred.get_series("DBAA", observation_start=start_date, observation_end=end_date)

  #V22
  df["TE1"] = df["DGS10"] - df["DTB4WK"]
  #V23
  df["TE2"] = df["DGS10"] - df["DTB3"]
  #V24
  df["TE3"] = df["DGS10"] - df["DTB6"]

  #V25
  df["TE5"] = df["DTB3"] - df["DTB4WK"]
  #V26
  df["TE6"] = df["DTB6"] - df["DTB4WK"]

  #V27
  df["DE1"] = df["DBAA"] - df["DAAA"] #Verify this - I believe there is a type in the paper ("BAAA")
  #V28
  df["DE2"] = df["DBAA"] - df["DGS10"]

  #V29
  df["DE4"] = df["DBAA"] - df["DTB6"]
  #V30
  df["DE5"] = df["DBAA"] - df["DTB3"]
  #V31
  df["DE6"] = df["DBAA"] - df["DTB4WK"]




ABOVE THIS LINE, make sure the label match the dataset, rather than the table

*V32 to V34*  
These values are not the same as the ones on their dataset

In [11]:
for index, df in data.items():
  df["CTB3M"] = fred.get_series('DGS3MO', observation_start=start_date, observation_end=end_date, units="pch")
  df["CTB6M"] = fred.get_series('DGS6MO', observation_start=start_date, observation_end=end_date, units="pch")
  df["CTB1Y"] = fred.get_series('DGS1', observation_start=start_date, observation_end=end_date, units="pch")

*V35 to V44*

In [ ]:
T1 = fred.get_series('DCOILBRENTEU', observation_start=go_back_n_days(start_date, 1), observation_end=end_date)
T2 = T1.fillna(method='ffill').diff()
T3 = T2.where(T1.notna())
print(T3.iloc[1:])

2009-12-31    0.29
2010-01-01     NaN
2010-01-04    1.14
2010-01-05    0.22
2010-01-06    0.87
              ... 
2012-11-26    0.04
2012-11-27   -0.85
2012-11-28   -1.07
2012-11-29    2.21
2012-11-30    0.37
Length: 762, dtype: float64


In [12]:
for index, df in data.items():
  #V35 - Value differ slightly from dataset
  T1 = fred.get_series('DCOILWTICO', observation_start=go_back_n_days(start_date, 1), observation_end=end_date)
  T2 = T1.fillna(method='ffill').diff()
  df["WIT-oil"] = T2.where(T1.notna())

  #V36 - Value differ slightly from dataset, in fact there is no "Brent-oil", just Brent - using fred instead of investing.com
  T1 = fred.get_series('DCOILWTICO', observation_start=go_back_n_days(start_date, 1), observation_end=end_date)
  T2 = T1.fillna(method='ffill').diff()
  df["Brent-oil"] = T2.where(T1.notna())

  #V37 Skipping due to redudancy

  #38



/tmp/ipykernel_19723/689674126.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  T2 = T1.fillna(method='ffill').diff()
/tmp/ipykernel_19723/689674126.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  T2 = T1.fillna(method='ffill').diff()
/tmp/ipykernel_19723/689674126.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  T2 = T1.fillna(method='ffill').diff()
/tmp/ipykernel_19723/689674126.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  T2 = T1.fillna(method='ffill').diff()
/tmp/ipykernel_19723/689674126.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill()

In [13]:
for index, df in data.items():
  for i in range(1, 83-41):
    df[f'Close-{i}'] = df['Close']

In [ ]:
print(len(data['^DJI'].columns))

82


**Primative Data Save**

In [20]:
datasetName = "tenTest"
datasetDir = "./Datasets"

os.makedirs(datasetDir, exist_ok=True)
datasetPath = join(datasetDir, datasetName) + ".pkl"


with open(datasetPath, 'wb') as file:
    pickle.dump(data, file)

**Pre-Processing Functions**

In [6]:
from sklearn.preprocessing import scale
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


#IMPORT FROM dataset.py CNNpred-pytorch
class WholeDataset(Dataset):

    def __init__(self, data, target):
        self.data = data
        self.target = target


    def __len__(self):
        return self.target.shape[0]

    def __getitem__(self, idx):
        #print("Accessing data at index", idx, "with shape:", self.data[idx].shape)
        return self.data[idx], self.target[idx]

#IMPORT FROM dataset.py CNNpred-pytorch
def generate_batches(dataset, #ONLY GENERATES TWO BATCHES
                     batch_size,
                     shuffle=True,
                     drop_last=False,
                     device="cpu",
                     n_workers=0):
    dataloader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=n_workers,
                            pin_memory=False)

    for data, labels in dataloader:
        #data = torch.unsqueeze(data, 1).float()
        data = data.float()
        labels = labels.float()
        yield data.to(device, non_blocking=True), labels.to(device, non_blocking=True)


#Personally I think this should be applied to the data before it is split into
#train, test, and val, but following CNNpred code, the test set losses seq_len
#points with this method
def create_windows(X, y, seq_len):
  series = []
  target = []
  for i in range(len(X) - seq_len + 1):
    series.append(X[i: i + seq_len])
    target.append(y[i + seq_len - 1])

  return np.array(series), np.array(target)


def preprocess(data: dict,
               Split_Date: str,
               seq_len: int,
               val_percent: float,
               n_day_predict: int = 1,
               StartTrim: int = 200) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
  """
    Parameters:
  - data (dict): Python dictionary that contains a dataframe for each stock
  - target (str): Stock that is being predicted, str must match data key name
  - Split_Date (str): The date (YYYY-MM-DD) that separates the training data from the testing data.
  - Seq_len (int): Time series length of the model input
  - val (float): percent of training devoted to validation
  - n_day_predict (int, optional): The number of days ahead to predict, default is 1.
  - StartTrim (int, optional): The number of initial rows to trim from the data for processing, default is 200.

  Returns:
  - Tuple containing:
    - X_train_np (np.ndarray): The feature matrix for the training set.
    - y_train (dict): The target dict for the training set. Key for each stock.
    - X_test_np (np.ndarray): The feature matrix for the testing set.
    - y_test (dict): The target dict for the testing set. Key for each stock.
    - X_val_np (np.ndarray): The feature matrix for the validation set.
    - y_val (dict): The target dict for the validation set. Key for each stock.


    TODO: Ability to make Validation set = training set
  """

  X_train = []

  X_test = []

  X_val = []

  y_train = {}

  y_test = {}

  y_val = {}

  for index, df in data.items():


    y = (df['Close'][n_day_predict:] / df['Close'][:-n_day_predict].values).astype(int)
    y = y[StartTrim - 1:]
    y = y[:-n_day_predict]

    #Split data
    y_train_i = y[y.index < Split_Date]

    y_test_i =  y[y.index >= Split_Date]


    #Validation split
    y_val_i = y_train_i[int((1-val_percent) * len(y_train_i)) - seq_len:]
    y_train_i = y_train_i[:int((1-val_percent) * len(y_train_i))]

    y_train[index] = y_train_i

    y_test[index] = y_test_i

    y_val[index] = y_val_i


    #Trim and fill
    df_pr = df[StartTrim:] #dataframe processed, first trim entries, 200 because of 200 day moving avg
    df_pr = df_pr.fillna(0)
    df_pr = df_pr[:-n_day_predict]
    df_pr.replace([np.inf, -np.inf], 0, inplace=True)


    #Split data
    train = df_pr[df_pr.index < Split_Date]
    test = df_pr[df_pr.index  >= Split_Date]

    #Validation split
    val = train[int((1-val_percent) * len(train)) - seq_len:]
    train = train[:int((1-val_percent) * len(train))]


    #Scale data and append
    X_train.append(scale(train))
    X_test.append(scale(test))
    X_val.append(scale(val))

  #Create Windows
  for index, y_train_i in y_train.items():
    X_train_np, y_train[index] = create_windows(np.transpose(np.array(X_train), (1, 0, 2)), np.array(y_train_i).flatten(), seq_len)

  for index, y_test_i in y_test.items():
    X_test_np,  y_test[index]  = create_windows(np.transpose(np.array(X_test), (1, 0, 2)), np.array(y_test_i).flatten(), seq_len)

  for index, y_val_i in y_val.items():
    X_val_np,   y_val[index]   = create_windows(np.transpose(np.array(X_val), (1, 0, 2)), np.array(y_val_i).flatten(), seq_len)
  return X_train_np, y_train, X_test_np,  y_test, X_val_np, y_val


**Model Definitions**

In [ ]:
"""
Notes on Default CNN model
Parameters:
seq_len(How far back are we using of time series data)
epoc
dropout ratio

Generate CNN ready data windows

for n times (They use 40)


CNN pred arch


Layer 1
Conv2d
filters: 8
stride: (1,1)
activation: relu
kernal: (1,1)
data_format: Channel_last
input shape: (# stocks, sequence length, # features)

Layer 2
Conv2d
filters: 8
stride: (1,1)
activation: relu
kernal: (# stocks, 3)

MaxPool2d
pool size (1, 2)

Layer 3
Conv2d
filters: 8
stride: (1,1)
activation: relu
kernal: (# stocks, 3)

MaxPool2d
pool size (1, 2)

Dropout 0.1

Dense
1, activation sigmoid

optimizer Adam, Loss= Mae, metrics Acc, F1
"""

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class CNNModel(nn.Module):
    def __init__(self, number_filter, number_of_stocks, seq_len, number_feature, drop):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, number_filter, kernel_size=(1, 1))
        self.conv2 = nn.Conv2d(number_filter, number_filter, kernel_size=(3, number_of_stocks))
        self.pool1 = nn.MaxPool2d((1, 2))
        self.conv3 = nn.Conv2d(number_filter, number_filter, kernel_size=(3, 1))
        self.pool2 = nn.MaxPool2d((1, 2))
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(drop)
        self.fc = nn.Linear(number_filter * (number_of_stocks * seq_len) // 4, 1)  # Adjust the size accordingly, should be 104 x 1 for 5x60x82

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc(x))
        return x

**Utility Functions** Might combine with preprocessing

In [8]:
#Use for deriving next model filename
def next_file(file, path):
  #Ensure dicectory exists
  os.makedirs(path, exist_ok=True)

  #Get root and extension
  filename, ext = os.path.splitext(file)

  #Get list of files
  files = os.listdir(path)
  num = []
  for file in files:
    if file.startswith(filename):
      num.append(int(re.findall(r'\d+', file.split('-')[-1])[0]))

  #Find next iteration number
  file_iteration_number = max(num) + 1 if len(num) else 1
  return join(path, filename + "-" + str(file_iteration_number) + ext)

**Engine Code**

In [ ]:
"""
CNNpred-pytorch imp
-Wand init (Optional)


-Get training file names


-'Construct_data_warehouse'


-'Transforrm_data_warehouse'


-CNN data sequences


-Create training, val, and test


-Train model i times

"""

In [ ]:
"""
General Architecture

Define Model
-Model Type and hyperparams
-optimizer
-loss function
-scheduler

preprocess data
-Get a y_train y_val y_test for each stock

>For each iter_num(Can be set to just 1)
  Directory Management
  -Get name of next model file name


  >For each stock
    Train
    -Take model params, and train
    -Record Acc, F1, Loss

  Record results
  -In an CSV in the results/<modelname>/, record the Loss, Acc, F1 for each Stock
  -In a 'Name' column, specify filename. i.e {Variant}-{iteration}
"""

In [9]:
#Main.py script
#Define Model and preprocessing params
config_path = "./Configs/crypto_test.yaml" #TODO: Define config file, this will be done in command line
with open(config_path, "r") as file:
  config = yaml.safe_load(file)

with open(config['preprocess']['dataset_path'], 'rb') as file:
  data = pickle.load(file)


preparams = {
    "data": data,
    "n_day_predict": config['preprocess']['n_day_predict'],
    "seq_len": config['preprocess']['seq_len'],
    "val_percent": config['preprocess']['val_percent'],
    "StartTrim" : config['preprocess']['StartTrim'],
    "Split_Date": str(config['preprocess']['Split_Date'])
}


In [11]:
#Main.py script
#preprocess data

X_train, y_train, X_test, y_test, X_val, y_val = preprocess(**preparams)

#Dict of Datasets
train_data = {}
val_data = {}
test_data = {}

#Dict of batch generaters
train_dataloader = {}

#Create wholedatasets in dict for each stock
#Create batch generator for training
print(X_train.shape)

X_val = X_val.transpose(0, 3, 1, 2) #TEMP
X_test = X_test.transpose(0, 3, 1, 2) #TEMP

X_train = X_train.transpose(0, 3, 1, 2) #TEMP
for index, y_train_i in y_train.items():
  train_data[index] = WholeDataset(X_train, y_train_i)
  # train_dataloader[index] = generate_batches(train_data[index], config['train']['batch_size'], config['train']['num_workers']) #TODO: Replace 128 and 0 with .yaml settings
  #Train_dataloader moved to training loop, as it needs to be re init every time

for index, y_val_i in y_val.items():

  val_data[index] = WholeDataset(X_val, y_val_i)

for index, y_test_i in y_test.items():
  test_data[index] = WholeDataset(X_test, y_test_i)

/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:247: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:247: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:247: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:247: UserWarning: Numeri

(1515, 60, 5, 82)


In [40]:
class CNNModelOne(nn.Module):
    def __init__(self, number_filter, number_of_stocks, seq_len, number_feature, drop, calculated_fc_layer_size):
        super(CNNModelOne, self).__init__()

        # Layer 1
        #self.conv1 = nn.Conv2d(in_channels=number_feature, out_channels=number_filter[0], kernel_size=(1, 1))
        self.conv1 = nn.Conv2d(in_channels=82, out_channels=8, kernel_size=(1, 1))
        # Layer 2
        #self.conv2 = nn.Conv2d(in_channels=number_filter[0], out_channels=number_filter[1], kernel_size=(1, 3))

        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 5))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1))

        # Layer 3
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 1))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 1))

        # Flatten and Dropout
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(drop)

        # Output layer
        self.fc = nn.Linear(calculated_fc_layer_size, 1)  # Calculate based on output size after last pooling

    def forward(self, x):
        """
        torch.Size([128, 82, 60, 5])
        torch.Size([128, 8, 60, 5])
        torch.Size([128, 8, 58, 1])
        torch.Size([128, 8, 29, 1])
        torch.Size([128, 8, 27, 1])
        torch.Size([128, 8, 13, 1])
        torch.Size([128, 104])
        torch.Size([128, 104])
        """
        #print("-----------------x-x-xxx--------------")
        #print(x.shape) # torch.Size([128, 82, 60, 5])
        x = F.relu(self.conv1(x))
        #print(x.shape) # torch.Size([128, 8, 60, 5])
        x = F.relu(self.conv2(x))
        #print(x.shape) # torch.Size([128, 8, 58, 1])
        x = self.pool1(x)
        #print(x.shape) # torch.Size([128, 8, 29, 1])
        x = F.relu(self.conv3(x))
        #print(x.shape) # torch.Size([128, 8, 27, 1])
        x = self.pool2(x)
        #print(x.shape) # torch.Size([128, 8, 13, 1])
        x = self.flatten(x)
        #print(x.shape) # torch.Size([128, 104])
        x = self.dropout(x)
        #print(x.shape) # torch.Size([128, 104])
        x = torch.sigmoid(self.fc(x))
        return x

In [41]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

#Train.py script
def validate(args, model, dataset):
    model.eval()
    loss_fcn = torch.nn.BCELoss()

    data_dataloader = generate_batches(dataset, args.batch_size, n_workers=args.num_workers)


    loss_list = []
    pred_list = []
    label_list = []
    with torch.no_grad():
        for batch_data, batch_label in data_dataloader:
            batch_logit = model(batch_data).view(-1)

            loss = loss_fcn(batch_logit, batch_label)

            pred = (batch_logit > 0.5).int()

            pred_list.extend(pred)
            label_list.extend(batch_label)

            loss_list.append(loss.item())

        loss_data = np.array(loss_list).mean()
        acc = accuracy(pred_list, label_list)
        f1 = f1_score(pred_list, label_list, average='macro')

    return loss_data, acc, f1,
def train(config, train_data):
  #Init Model
  #model_class = getattr(models, config['model']['type'])
  #model = model_class(**config['model']['params'])

  ####DEBUG DEBUG DEBUG
  model = CNNModelOne(number_filter=[8,8,8], number_of_stocks=5, seq_len=60, number_feature=82, drop=0.1, calculated_fc_layer_size =104)
  # print("-------Model----------")
  # print(model)
  # print(model.conv3.kernel_size)
  # print(model.conv3.padding)
  # print("-------Model----------")

  #Init loss function
  #loss_fcn = torch.nn.BCELoss()
  loss_class = getattr(nn, config['loss_function']['type'])
  loss_fcn = loss_class(**config['loss_function']['params'])

  #Init Optimizer
  #optimizer_class = getattr(optim, config['optimizer']['type'])
  #optimizer = optimizer_class(model.parameters(), **{k: v for k, v in config['optimizer'].items() if k != 'type'})


  ###DEBUG DEBUG DEBUG
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=128)




  #Init Scheduler
  #scheduler_class = getattr(optim.lr_scheduler, config['scheduler']['type'])
  #scheduler = scheduler_class(optimizer, **{k: v for k, v in config['scheduler'].items() if k != 'type'})

  ###DEBUG DEBUG DEBUG
  scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=True, eps=1e-8)
  for epoch in range(config['train']['epoch']):
    model.eval()
    loss_list = []
    pred_list = []
    label_list = []
    #train_dataloader
    train_dataloader = generate_batches(train_data, config['train']['batch_size'], config['train']['num_workers'])

    for batch_data, batch_label in train_dataloader:
      # print("-------bd----------")
      # print(batch_data.shape)
      # print("-------------------")
      batch_logit = model(batch_data).view(-1)

      loss = loss_fcn(batch_logit, batch_label)

      # print("Batch_logit")
      # print(batch_logit)

      pred = (batch_logit > 0.5).int()
      # print("Pred")
      # print(pred)

      pred_list.extend(pred)
      label_list.extend(batch_label)

      optimizer.zero_grad()
      loss.backward()
      parameters = list(model.parameters())
      optimizer.step()
      loss_list.append(loss.item())

    loss_data = np.array(loss_list).mean()

    train_acc = accuracy(pred_list, label_list)
    # print("-------acc----------")
    # print(train_acc)
    # print(pred_list)
    # print(label_list)
    # print("-------acc----------")

    train_f1 = f1_score(pred_list, label_list, average='macro')

    print("Epoch {:05d}\n"
          "Train: loss: {:.4f} | accuracy: {:.4f} | f-acore: {:.4f}"
          .format(epoch + 1, loss_data, train_acc, train_f1))
    #loss_data = float(loss_data.item())

    loss_data = torch.tensor(loss_data)
    scheduler.step(loss_data)

    # if (epoch + 1) % 1 == 0:
    #   val_loss, val_acc, val_f1 = validate(args, model, val_dataset)
    #   print("Validation:  loss: {:.4f} | accuracy: {:.4f} | f1: {:.4f}"
    #         .format(val_loss, val_acc, val_f1))
    #   # choosing best model according to best validation accuracy
    #   if best_f1 < val_f1:
    #       best_f1 = val_f1
    #       cur_step = 0
    #       torch.save(model, filepath)

      # else:
      #     cur_step += 1
      #     if cur_step == config['train']['patience']:
      #         break

In [42]:
#Main.py script
#Training loop
import models #TODO: Put this at top of imports


gen_num = 1 #TODO: Define this in the Config file
for iteration in range(gen_num):


  for index, _ in y_train.items():
    #Get Model output path
    model_path = next_file(config['model']['type'] + index, join("./Results/", os.path.basename(config_path)))
    # for b, a, in train_dataloader[index]:
    #   print("opppppppppppppppppppppppppppppppppppppppppppppppppppppppp")
    train(config, train_data[index])


/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00001
Train: loss: 0.6988 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00002
Train: loss: 0.6986 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00003
Train: loss: 0.6983 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00004
Train: loss: 0.6981 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00005
Train: loss: 0.6979 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00006
Train: loss: 0.6977 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00007
Train: loss: 0.6975 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00008
Train: loss: 0.6973 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00009
Train: loss: 0.6971 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00010
Train: loss: 0.6970 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00011
Train: loss: 0.6968 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00012
Train: loss: 0.6967 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00013
Train: loss: 0.6965 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00014
Train: loss: 0.6964 | accuracy: 0.4825 | f-acore: 0.3255
Epoch 00015
Train: loss: 0.6963 | 

/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00001
Train: loss: 0.6926 | accuracy: 0.5274 | f-acore: 0.5265
Epoch 00002
Train: loss: 0.6925 | accuracy: 0.5281 | f-acore: 0.5280
Epoch 00003
Train: loss: 0.6925 | accuracy: 0.5261 | f-acore: 0.5254
Epoch 00004
Train: loss: 0.6925 | accuracy: 0.5182 | f-acore: 0.5148
Epoch 00005
Train: loss: 0.6925 | accuracy: 0.5168 | f-acore: 0.5067
Epoch 00006
Train: loss: 0.6925 | accuracy: 0.5234 | f-acore: 0.5062
Epoch 00007
Train: loss: 0.6925 | accuracy: 0.5195 | f-acore: 0.4925
Epoch 00008
Train: loss: 0.6925 | accuracy: 0.5116 | f-acore: 0.4719
Epoch 00009
Train: loss: 0.6925 | accuracy: 0.5188 | f-acore: 0.4651
Epoch 00010
Train: loss: 0.6925 | accuracy: 0.5215 | f-acore: 0.4547
Epoch 00011
Train: loss: 0.6925 | accuracy: 0.5248 | f-acore: 0.4467
Epoch 00012
Train: loss: 0.6925 | accuracy: 0.5208 | f-acore: 0.4306
Epoch 00013
Train: loss: 0.6926 | accuracy: 0.5188 | f-acore: 0.4111
Epoch 00014
Train: loss: 0.6926 | accuracy: 0.5162 | f-acore: 0.4003
Epoch 00015
Train: loss: 0.6926 | 

/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00001
Train: loss: 0.6932 | accuracy: 0.5056 | f-acore: 0.5027
Epoch 00002
Train: loss: 0.6931 | accuracy: 0.5083 | f-acore: 0.5074
Epoch 00003
Train: loss: 0.6931 | accuracy: 0.5096 | f-acore: 0.5086
Epoch 00004
Train: loss: 0.6931 | accuracy: 0.5056 | f-acore: 0.5044
Epoch 00005
Train: loss: 0.6931 | accuracy: 0.5083 | f-acore: 0.5074
Epoch 00006
Train: loss: 0.6931 | accuracy: 0.5069 | f-acore: 0.5063
Epoch 00007
Train: loss: 0.6931 | accuracy: 0.5096 | f-acore: 0.5089
Epoch 00008
Train: loss: 0.6931 | accuracy: 0.5089 | f-acore: 0.5083
Epoch 00009
Train: loss: 0.6931 | accuracy: 0.5142 | f-acore: 0.5137
Epoch 00010
Train: loss: 0.6931 | accuracy: 0.5168 | f-acore: 0.5163
Epoch 00011
Train: loss: 0.6931 | accuracy: 0.5168 | f-acore: 0.5164
Epoch 00012
Train: loss: 0.6931 | accuracy: 0.5122 | f-acore: 0.5119
Epoch 00013
Train: loss: 0.6931 | accuracy: 0.5083 | f-acore: 0.5079
Epoch 00014
Train: loss: 0.6931 | accuracy: 0.5076 | f-acore: 0.5072
Epoch 00015
Train: loss: 0.6931 | 

/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00001
Train: loss: 0.6933 | accuracy: 0.4944 | f-acore: 0.4828
Epoch 00002
Train: loss: 0.6933 | accuracy: 0.4957 | f-acore: 0.4815
Epoch 00003
Train: loss: 0.6933 | accuracy: 0.4983 | f-acore: 0.4810
Epoch 00004
Train: loss: 0.6933 | accuracy: 0.4983 | f-acore: 0.4791
Epoch 00005
Train: loss: 0.6933 | accuracy: 0.5023 | f-acore: 0.4801
Epoch 00006
Train: loss: 0.6933 | accuracy: 0.5063 | f-acore: 0.4815
Epoch 00007
Train: loss: 0.6933 | accuracy: 0.5043 | f-acore: 0.4783
Epoch 00008
Train: loss: 0.6932 | accuracy: 0.5043 | f-acore: 0.4773
Epoch 00009
Train: loss: 0.6932 | accuracy: 0.5056 | f-acore: 0.4774
Epoch 00010
Train: loss: 0.6932 | accuracy: 0.5056 | f-acore: 0.4760
Epoch 00011
Train: loss: 0.6932 | accuracy: 0.5089 | f-acore: 0.4769
Epoch 00012
Train: loss: 0.6932 | accuracy: 0.5069 | f-acore: 0.4716
Epoch 00013
Train: loss: 0.6932 | accuracy: 0.5050 | f-acore: 0.4673
Epoch 00014
Train: loss: 0.6932 | accuracy: 0.5056 | f-acore: 0.4653
Epoch 00015
Train: loss: 0.6932 | 

/home/drew/anaconda3/envs/CNNpredRep/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 00001
Train: loss: 0.6931 | accuracy: 0.5168 | f-acore: 0.5027
Epoch 00002
Train: loss: 0.6930 | accuracy: 0.5155 | f-acore: 0.4961
Epoch 00003
Train: loss: 0.6930 | accuracy: 0.5129 | f-acore: 0.4888
Epoch 00004
Train: loss: 0.6930 | accuracy: 0.5142 | f-acore: 0.4843
Epoch 00005
Train: loss: 0.6929 | accuracy: 0.5168 | f-acore: 0.4831
Epoch 00006
Train: loss: 0.6929 | accuracy: 0.5188 | f-acore: 0.4804
Epoch 00007
Train: loss: 0.6929 | accuracy: 0.5195 | f-acore: 0.4763
Epoch 00008
Train: loss: 0.6929 | accuracy: 0.5188 | f-acore: 0.4709
Epoch 00009
Train: loss: 0.6929 | accuracy: 0.5162 | f-acore: 0.4612
Epoch 00010
Train: loss: 0.6929 | accuracy: 0.5122 | f-acore: 0.4515
Epoch 00011
Train: loss: 0.6929 | accuracy: 0.5089 | f-acore: 0.4419
Epoch 00012
Train: loss: 0.6929 | accuracy: 0.5083 | f-acore: 0.4367
Epoch 00013
Train: loss: 0.6929 | accuracy: 0.5076 | f-acore: 0.4306
Epoch 00014
Train: loss: 0.6929 | accuracy: 0.5135 | f-acore: 0.4327
Epoch 00015
Train: loss: 0.6929 | 